In [369]:
import json
import urllib
import pandas as pd
import numpy as np
import requests
import re
import matplotlib.pyplot as plt
from scrapy.selector import Selector
from selenium import webdriver
from yellowbrick.text import FreqDistVisualizer
import os
from time import sleep
from bs4 import BeautifulSoup
%matplotlib inline
driver = webdriver.Chrome(executable_path="./chromedriver")

In [265]:
driver.get('http://google.com')

In [145]:
'''
path & datastorage
'''
scraping_start_url='https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk='
cols=['job_title','company','job_post_date','vjk','job_description_p','job_description_li','job_description_all']
job_title_no_sal_list=[]
job_title_sal_list=[]
company_no_sal_list=[]
company_sal_list=[]
post_date_no_sal_list=[]
post_date_sal_list=[]
review_no_sal_list=[]
review_sal_list=[]
JD_p_no_sal_list=[]
JD_p_sal_list=[]
JD_li_no_sal_list=[]
JD_li_sal_list=[]
JD_all_no_sal_list=[]
JD_all_sal_list=[]
df_job_no_sal=pd.DataFrame(columns=cols)
df_job_sal=pd.DataFrame(columns=cols)
df_review_no_sal=pd.DataFrame(columns=['vjk_id','review_counts'])
df_review_sal=pd.DataFrame(columns=['vjk_id','review_counts'])

# Using Xpath helper and development toolsets as well as random sampling and browsing several job windows, I decided to do web scraping on data related job with salary and data related job without salary information for  raw data first

## Targeting data:
### vjk ID to address job window, job title, company, post date, review, 3 possible job details and salary, just FYI, salary I will extract by post processing as the salary data randomly appered everywhere [Due to indeed horrible html with java structure (which can also tell the benefits of API)]. I will use the ratio of [logged data/all data] to check the successful rate of web scraping. The target is to cover as much cases as possible. Below are examples of my testing entry detection and examination.

#### The very last portion is example codes <- using single data entry for demonstration purpose

In [ ]:
'''
Example of non-salary url, start=0,10,20,30...(page 1,2,3...)
'''

"https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0"

'''
Example of salary url
'''
'https://www.indeed.com.sg/jobs?q=data%20%244000&l=Singapore&start=20&vjk=c44f7fb7b3577a8e'
'https://www.indeed.com.sg/data-$4000-jobs-in-Singapore'#this format is easier
'''
Detailed job page is formed by:
https://www.indeed.com.sg/jobs?q=data%20%244000&l=Singapore&start=20&vjk=0bde816b5390f14f
                                          !4000 here means salary upper-bound is 4000
                                                           !sstart=20 here means 3rd web page
                                                                    !vjk=0bde... here means specific job description
'''

# Query Data

In [589]:
'''
No Salary
'''

def indeed_scraping(vjk_lis,df,df_review,jt=[],comp=[],date=[],review=[],jd_p=[],jd_li=[],jd_all=[],url=scraping_start_url):
    for vjk_id in vjk_lis:
        scraping_finish_url = url + vjk_id
        print(scraping_finish_url)
        dr=webdriver.Chrome(executable_path="./chromedriver")
        dr.get(scraping_finish_url)
        wait=random.randint(12,25)
        sleep(wait)
        html = dr.page_source.lower().strip()
        dr.close()
        
        soup = BeautifulSoup(html, 'lxml')
        
        #Job_title
        for title in soup.findAll('div',{'id':'vjs-jobtitle'}):
            jt.append(title.text)
        
        #company
        for company in soup.findAll('span',{'id':'vjs-cn'}):
            comp.append(company.text)
        
        #post date
        temp_date=''
        for post_date in soup.findAll('span',{'class':'date'}):
            temp_date=post_date.text
        date.append(temp_date)
        
        #review
        #temp_vjk_id=[]
        #temp_review=[]
        #temp_df=pd.DataFrame(columns=['vjk_id','review_counts'])
        #for x in soup.findAll('a',{'class':'turnstilelink slnounderline '}):
            #temp_vjk_id.append(str(x).split('fromjk=')[1].split('&')[0])
            #temp_review.append(x.text.strip())
        #temp_df['vjk_id']=temp_vjk_id
        #temp_df['review_counts']=temp_review
        #df_review=df_review.append(temp_df,ignore_index=True)
        #df_review['vjk_id']=pd.concat([df_review['vjk_id'],pd.Series(temp_vjk_id)])
        #df_review['review_counts']=pd.concat([df_review['review_counts'],pd.Series(temp_review)])  
        
        #JD_p
        for x in soup.findAll('div',{'id':"vjs-desc"}):
            job_des_p=''
            for y in x.findAll('p'):
                job_des_p+=","
                job_des_p+=y.text.strip()
            jd_p.append(job_des_p)
        
        #JD_li
        for x in soup.findAll('div',{'id':"vjs-desc"}):
            job_des_li=''
            for y in x.findAll('li'):
                job_des_li+=","
                job_des_li+=y.text.strip()
            jd_li.append(job_des_li)
        
        #JD_all
        for x in soup.findAll('div',{'id':"vjs-desc"}):
            job_des_all=''
            job_des_all=x.text.strip()
            jd_all.append(job_des_all)
    
    #print(len(jt),jt)
    #print(len(comp),comp)
    #print(len(date),date)
    #print(len(vjk_lis),vjk_lis)
    #print(len(jd_p),jd_p)
    #print(len(jd_li),jd_li)
    #print(len(jd_all),jd_all)
    
    df_job_temp=pd.DataFrame(columns=cols)
    df_job_temp[cols[0]]=pd.Series(jt)
    df_job_temp[cols[1]]=pd.Series(comp)
    df_job_temp[cols[2]]=pd.Series(date)
    df_job_temp[cols[3]]=pd.Series(vjk_lis)
    df_job_temp[cols[4]]=pd.Series(jd_p)
    df_job_temp[cols[5]]=pd.Series(jd_li)
    df_job_temp[cols[6]]=pd.Series(jd_all)
    df=df.append(df_job_temp,ignore_index=True)
    return df
    #print(df)
    #print('-'*80)
    #df[cols[0]]=pd.concat([df[cols[0]],pd.Series(jt)])
    #df[cols[1]]=pd.concat([df[cols[1]],pd.Series(comp)])
    #df[cols[2]]=pd.concat([df[cols[2]],pd.Series(date)])
    #df[cols[3]]=pd.concat([df[cols[3]],pd.Series(vjk_lis)])
    #df[cols[4]]=pd.concat([df[cols[4]],pd.Series(jd_p)])
    #df[cols[5]]=pd.concat([df[cols[5]],pd.Series(jd_li)])
    #df[cols[6]]=pd.concat([df[cols[6]],pd.Series(jd_all)])

In [81]:
#df_review_sal['vjk_id']=[1,2,3,4,5]
#df_review_sal['review_counts']=[10,20,30,40,50]

In [82]:
#chk=pd.DataFrame()

In [85]:
#chk=chk.append(df_review_sal,ignore_index=True)

# vjk No Salary

In [274]:
no_sal_page_list=[page for page in range(500,1000,10)] #100 pages
#no_sal_page_list=[page for page in range(0,30,10)] #5 pages
url_no_sal_start='https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=' #baseline
vjk_no_sal_list=[]#list to hold vjk job key
for page in no_sal_page_list:
    print(page)
    url_no_sal_finish=url_no_sal_start+str(page)
    print(url_no_sal_finish)
    response = requests.get(url_no_sal_finish)
    sleep(2)
    HTML=response.text.lower()
    print(HTML[:50])
    vjk_no_sal_list.extend(Selector(text=HTML).xpath('//div/@data-jk').extract())
    print(Selector(text=HTML).xpath('//div/@data-jk').extract())
    df_job_no_sal=indeed_scraping(Selector(text=HTML).xpath('//div/@data-jk').extract(),
                                  df=df_job_no_sal,
                                  df_review=df_review_no_sal)    

500
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=500
<!doctype html>
<html lang="en" dir="ltr">
<head>

['81ba4ee03fbdc88f', 'b2b7f1fd74fc0fdf', '6d7f5bd4600ae3fc', '1ceddf1fe1e8f2b6', '211a82f9901fa81f', 'f32ab4b5fe257eca', 'b30aff3cf0d80fc1', '1e4d46bd6af0cc73', '527b6e72e7ad20cb', 'd1dfa932021b9c92', '967e8983bfcba209', '79aa16e208108f78', 'aab0218907238930', '9656c51b6a118a89', '75aa081953111ad0']
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=81ba4ee03fbdc88f


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=61731): Max retries exceeded with url: /session/a5952b2582e00efb80d57df2e0c885e0/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a20bf3e10>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [142]:
df_job_no_sal

,job_title,company,job_post_date,vjk,job_description_p,job_description_li,job_description_all


# vjk With Salary

In [590]:
sal_list=[salary for salary in range(4000,12200,200)]
url_sal_start='https://www.indeed.com.sg/data-$%s-jobs-in-Singapore'
vjk_sal_list=[]
for salary in sal_list:
    url_sal_finish=url_sal_start%salary
    response = requests.get(url_sal_finish)
    wait=random.randint(5,10)
    sleep(1)
    HTML=response.text.lower()
    vjk_sal_list.extend(Selector(text=HTML).xpath('//div/@data-jk').extract())
    df_job_sal=indeed_scraping(Selector(text=HTML).xpath('//div/@data-jk').extract(),df=df_job_sal,df_review=df_review_sal)

https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=46a2c7296e551f66
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=17f54c1c4a78c20b
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=318af4b76afd6d18
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=7483d5e8a6e5e695
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=615f20279c5ebd0d
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=49b2d57c6e15b4eb
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=a826021e81ae5a28
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=eb2f9735e412802b
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=bacf99afc1d84e74
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=baa086c652a7acad
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=54ec636307175dbd
https://www.indeed.com.sg/jobs?q

https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=27e95baaf69125b2
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=5d17562c73786990
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=9bc65a2f86b8913d
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=7f5b423909f79810
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=16e88921d892b2bb
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=36ae67bf30a728bd
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=bf3120ed476883ac
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=14707ed993d8e052
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=6c434d2bdbaa78db
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=a2d9a066c990522a
https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=a61a2f36be6791e4
https://www.indeed.com.sg/jobs?q

In [592]:
df_job_sal.drop_duplicates()

,job_title,company,job_post_date,vjk,job_description_p,job_description_li,job_description_all
0,"credit control specialist ($4000, central)",kelly services singapore,21 days ago,46a2c7296e551f66,",the person will supports several post-sales f...",",minimum diploma in any field,minimum two-year...",minimum diploma in any field\nminimum two-year...
1,"junior supply planner / project (central, $4000)",kelly services singapore,18 days ago,17f54c1c4a78c20b,",requirements:,kelly services (singapore) pte ...",",working location at central, outside of cbd,5...","working location at central, outside of cbd\n5..."
2,business analyst (junior) / salary $4000 / mnc...,kelly services singapore,12 days ago,318af4b76afd6d18,",my client is a us multinational courier deliv...",",process, analyze, update requests for changes...",my client is a us multinational courier delive...
3,advanced data analyst (#fintech #startup),canopy pte ltd,30+ days ago,7483d5e8a6e5e695,",company,canopy is a ""make sense of your data""...",",bachelors in computer science / information t...","company\ncanopy is a ""make sense of your data""..."
4,site engineer / 5.5 work week,job alpha associates,6 days ago,615f20279c5ebd0d,",industry: construction / mnc,location: marin...",",diploma graduate from singapore institution (...",industry: construction / mnclocation: marine ...
5,freelance surveyor (up to $4000/o levels & up),bgc group pte ltd,8 days ago,49b2d57c6e15b4eb,",freelance surveyor (up to $4000/o-levels & up...",",to do door-to-door surveyor pertaining to the...",freelance surveyor (up to $4000/o-levels & up)...
6,computer vision software engineer,trakomatic pte. ltd.,8 days ago,a826021e81ae5a28,",trakomatic is one of south east asia's fastes...",",develop user-centric intuitive software to be...",trakomatic is one of south east asia's fastest...
7,helpdesk data analyst,chandler macleod,28 days ago,eb2f9735e412802b,",are you customer-service oriented with prior ...",",1 year contract,$4000 monthly, great office f...","1 year contract\n$4000 monthly, great office f..."
8,"it support engineer, as400 (5 day / central / ...",mci career services pte ltd,1 day ago,bacf99afc1d84e74,",responsibilities:,additional information:,app...",",provide erp support and troubleshooting for t...",responsibilities: provide erp support and trou...
9,programmer,prosper employment agency(ea lic: 08c5590),21 days ago,baa086c652a7acad,",responsibilities,requirements,working hours: ...",",develop programs to control hardware device u...",responsibilitiesdevelop programs to control ha...


In [593]:
df_job_sal.drop_duplicates().to_csv('indeed_with_salary.csv')

In [371]:
#print(len(vjk_sal_list))
#print(len(np.unique(vjk_sal_list)))
vjk_sal_unique=np.unique(vjk_sal_list)

119
101


In [373]:
'''
455ba066249003a7: Project Engineer $5000
555f853f0169a061: Product Manager $5000-$8000
aaf5206f215b6182: Business Intelligence / Data Analyst $3000-5000
'''
repeated=[x for x in vjk_sal_unique if x in vjk_no_sal_unique] #3 job appeared in both salary and no salary list

In [374]:
vjk_no_sal_unique=[x for x in vjk_no_sal_unique if x not in repeated]
#print(len(vjk_no_sal_unique))

# Query Data 

In [87]:
str_3=[]
for i in jj['data'][0]['links']['country']['languages'].keys():
    str_3.append(jj['data'][0]['links']['country']['languages'][i])
#jj['data'][0]['links']['country']['languages'].keys()

In [88]:
str_3

['Mandarin', 'English', 'Malay', 'Tamil']

In [34]:
len(jj['data'])

291

In [557]:
jj['data'][3]

{'reviewedDate': '2018-10-25T00:00:00.000Z',
 'id': '8162201f-2c64-4111-b11b-35e0a30938d9',
 'title': 'Data Science Consultant',
 'status': 'OPEN',
 'deadline': None,
 'startDate': None,
 'endDate': None,
 'bannerPic': None,
 'intro': None,
 'location': None,
 'type': 'FULL_TIME',
 'durationLegacy': None,
 'salaryLegacy': None,
 'expiryDate': '2018-11-25',
 'minYearsOfExperience': 1,
 'maxYearsOfExperience': 3,
 'attachments': [],
 'draftId': None,
 'isCoverLetterMandatory': False,
 'isPublic': True,
 'numberOfHires': 2,
 'isRemote': False,
 'descriptionRaw': {'blocks': [{'key': '2f51q',
    'data': {},
    'text': 'About you:',
    'type': 'unstyled',
    'depth': 0,
    'entityRanges': [],
    'inlineStyleRanges': [{'style': 'BOLD', 'length': 10, 'offset': 0},
     {'style': 'UNDERLINE', 'length': 10, 'offset': 0}]},
   {'key': '7bnq4',
    'data': {},
    'text': ' You have a gift for taking unstructured and messy data, cleaning it, finding key trends and outliers, developing predic

In [60]:
str_2=''
for i in range(len(jj['data'][1]['links']['company']['descriptionRaw']['blocks'])):
    str_2+=jj['data'][1]['links']['company']['descriptionRaw']['blocks'][i]['text'].strip()

In [61]:
str_2

"Taralite is a startup providing financial technology services in the area of money loan for the unbankables. A study has shown that only 60 millions out of 250-million Indonesians have bank accounts and out of those 60 million, only some are eligible to apply for a loan from a financial institution. And that's why we're here.We give that majority of Indonesians access to affordable loans so they are able to live up to their potentials and enjoy memorable moments in life, such as education. With interest rate from 0.99% each month, we serve people all across Indonesia and revolutionize Indonesia's underdeveloped loan markets."

In [418]:
len(JD_all_sal_list)

101

In [451]:
indeed_scraping(vjk_sal_unique,job_title_sal_list,company_sal_list,post_date_sal_list,review_sal_list,JD_p_sal_list,JD_li_sal_list,JD_all_sal_list,df=df_job_sal)

In [7]:
chk=requests.get('https://glints.com/api/jobs?where=%7B%22type%22:%7B%22in%22:[%22FULL_TIME%22]%7D,%22isPublic%22:true,%22isRemote%22:false,%22status%22:%22OPEN%22%7D&SearchTerm=data&include=JobSalaries,+City,+Country,+Company&order=updatedAt+DESC&limit=1000&offset=0')
sleep(1)

In [8]:
jj=json.loads(chk.text)

In [9]:
jj.keys()

dict_keys(['links', 'data', 'count'])

In [ ]:
web=requests.get('https://glints.com/api/jobs?where=%7B%22type%22:%7B%22in%22:[%22FULL_TIME%22]%7D,%22isPublic%22:true,%22isRemote%22:false,%22status%22:%22OPEN%22%7D&SearchTerm=data&include=JobSalaries,+City,+Country,+Company&order=updatedAt+DESC&limit=1000&offset=0')
sleep(1)
glints_json=json.loads(web.text)

In [172]:
title=[]
intro=[]
job_type=[]
min_y_exp=[]
max_y_exp=[]
is_public=[]
num_hire=[]
is_remote=[]
job_description=[]
country_code=[]
job_sal_mode=[]
job_sal_min=[]
job_sal_max=[]
sal_currency_code=[]
comp_name=[]
comp_tagline=[]
is_verified=[]
is_hiring=[]
lcal_office=[]
comp_description=[]
country_name=[]
country_demonym=[]
country_currency=[]
country_lang=[]
for counter in range(len(glints_json['data'])):
    title.append(glints_json['data'][counter]['title'])
    intro.append(glints_json['data'][counter]['intro'])
    job_type.append(glints_json['data'][counter]['type'])
    min_y_exp.append(glints_json['data'][counter]['minYearsOfExperience'])
    max_y_exp.append(glints_json['data'][counter]['maxYearsOfExperience'])
    is_public.append(glints_json['data'][counter]['isPublic'])
    num_hire.append(glints_json['data'][counter]['numberOfHires'])
    is_remote.append(glints_json['data'][counter]['isRemote'])
    
    str_1=''
    for ind in range(len(glints_json['data'][counter]['descriptionRaw']['blocks'])):
        str_1+=glints_json['data'][counter]['descriptionRaw']['blocks'][ind]['text'].strip()
    job_description.append(str_1)
    
    country_code.append(glints_json['data'][counter]['CountryCode'])
    
    try:
        job_sal_mode.append(glints_json['data'][counter]['links']['jobSalaries'][0]['salaryMode'])
    except:
        job_sal_mode.append(None)
        
    try:
        job_sal_min.append(glints_json['data'][counter]['links']['jobSalaries'][0]['minAmount'])
    except:
        job_sal_min.append(None)
        
    try:
        job_sal_max.append(glints_json['data'][counter]['links']['jobSalaries'][0]['maxAmount'])
    except:
        job_sal_max.append(None)
        
    try:
        sal_currency_code.append(glints_json['data'][counter]['links']['jobSalaries'][0]['CurrencyCode'])
    except:
        sal_currency_code.append(None)
    
    try:
        comp_name.append(glints_json['data'][counter]['links']['company']['name'])
    except:
        comp_name.append(None)
    
    try:
        comp_tagline.append(glints_json['data'][counter]['links']['company']['tagline'])
    except:
        comp_tagline.append(None)
    
    try:
        is_verified.append(glints_json['data'][counter]['links']['company']['isVerified'])
    except:
        is_verified.append(None)
        
    try:
        is_hiring.append(glints_json['data'][counter]['links']['company']['isHiring'])
    except:
        is_hiring.append(None)
        
    try:
        lcal_office.append(glints_json['data'][counter]['links']['company']['localOffice'])
    except:
        lcal_office.append(None)
        
    str_2=''
    try:
        for ind in range(len(glints_json['data'][counter]['links']['company']['descriptionRaw']['blocks'])):
            str_2+=glints_json['data'][counter]['links']['company']['descriptionRaw']['blocks'][ind]['text'].strip()
    except:
        str_2=None
    comp_description.append(str_2)
    
    country_name.append(glints_json['data'][counter]['links']['country']['name'])
    country_demonym.append(glints_json['data'][counter]['links']['country']['demonym'])
    country_currency.append(glints_json['data'][counter]['links']['country']['currency'])
    
    temp_lang_list=[]
    for key in glints_json['data'][counter]['links']['country']['languages'].keys():
        temp_lang_list.append(glints_json['data'][counter]['links']['country']['languages'][key])
    country_lang.append(temp_lang_list)

In [177]:
#glints_json['data'][2]['links']['country']

In [178]:
col_job=['title','intro','type','min_year_exp','max_year_exp','is_public','num_of_hires','is_remote','job_description','country_code','sal_mode','sal_min','sal_max','sal_currency_code']
col_comp=['comp_name','comp_tagline','comp_verification','comp_is_hiring','comp_local_office','comp_description']
col_country=['country_name','country_demonym','country_currency','country_language']
col_all=col_job+col_comp+col_country
df_glints_data=pd.DataFrame(columns=col_all)

In [180]:
df_glints_data[col_all[0]]=title
df_glints_data[col_all[1]]=intro
df_glints_data[col_all[2]]=job_type
df_glints_data[col_all[3]]=min_y_exp
df_glints_data[col_all[4]]=max_y_exp
df_glints_data[col_all[5]]=is_public
df_glints_data[col_all[6]]=num_hire
df_glints_data[col_all[7]]=is_remote
df_glints_data[col_all[8]]=job_description
df_glints_data[col_all[9]]=country_code
df_glints_data[col_all[10]]=job_sal_mode
df_glints_data[col_all[11]]=job_sal_min
df_glints_data[col_all[12]]=job_sal_max
df_glints_data[col_all[13]]=sal_currency_code
df_glints_data[col_all[14]]=comp_name
df_glints_data[col_all[15]]=comp_tagline
df_glints_data[col_all[16]]=is_verified
df_glints_data[col_all[17]]=is_hiring
df_glints_data[col_all[18]]=lcal_office
df_glints_data[col_all[19]]=comp_description
df_glints_data[col_all[20]]=country_name
df_glints_data[col_all[21]]=country_demonym
df_glints_data[col_all[22]]=country_currency
df_glints_data[col_all[23]]=country_lang

In [183]:
df_glints_data.to_csv('glints_data_science_job.csv')

In [ ]:
'''
'title','intro', 'type', 'minYearsOfExperience','maxYearsOfExperience', 'isPublic'
'numberOfHires', 'isRemote'

jobs count= len(jj['data']) <=c

'descriptionRaw':
    'blocks'
    
    counter: len(jj['data'][1]['descriptionRaw']['blocks'])
    
    for i in range(len(jj['data'][c]['descriptionRaw']['blocks'])):
    str_1+=jj['data'][c]['descriptionRaw']['blocks'][i]['text'].strip()

'CountryCode'

jj['data'][c]['links']:
    'jobSalaries'
        'salaryMode'
        'minAmount'
        'maxAmount'
        'CurrencyCode'
    'company':
        'name'
        'tagline'
        'isVerified'
        'isHiring'
        'localOffice'
        'descriptionRaw'
            str_2=''
            for i in range(len(jj['data'][c]['links']['company']['descriptionRaw']['blocks'])):
                str_2+=jj['data'][c]['links']['company']['descriptionRaw']['blocks'][i]['text'].strip()
    'country':
        'name'
        'demonym'
        'currency'
        'languages':
            str_3=[]
            for i in jj['data'][c]['links']['country']['languages'].keys():
                str_3.append(jj['data'][c]['links']['country']['languages'][i])    
'''

In [ ]:
job_title=[]
job_intro=[]
job_type_=[]
job_minexp=[]
job_maxexp=[]
job_is_public=[]
job_noh=[]
job_is_remote=[]

job_descriptionRaw_jd=[]
job_country_code=[]

job_link_sal_mode=[]
job_link_sal_min_sal=[]
job_link_sal_max_sal=[]
job_link_sal_currency=[]
job_link_comp_name=[]
job_link_comp_tag=[]
job_link_comp_verification=[]
job_link_comp_hiring=[]
job_link_comp_local_office=[]
job_link_comp_descript=[]
job_link_country_name=[]
job_link_country_demonym=[]
job_link_country_currency=[]
job_link_country_lang=[]

for count in len(jj['data']):
    job_title.append(jj['data'][count]['title'])
    job_intro.append(jj['data'][count]['intro'])
    job_type_.append(jj['data'][count]['type'])
    

In [92]:
jj['data'][1]['intro']

In [21]:
len(jj['data'][1]['descriptionRaw']['blocks'])

10

In [48]:
str_1=''
for i in range(len(jj['data'][1]['descriptionRaw']['blocks'])):
    str_1+=jj['data'][1]['descriptionRaw']['blocks'][i]['text'].strip()


In [49]:
str_1.strip()

'Job Description:Assist to create fast, accurate, robust and scalable data-processing in our companyDevelop new features together with our team of data scientist and software developerBuild data computational pipelines and ETLsPreferred Qualifications:Enrolled in an undergraduate or graduate degree from top universities in a quantitative discipline preferredYou have prior experience with python, SQL, NoSQLLinux scripting is a plusFamiliar with REST-APIs using Python or similar technologies'

In [226]:
kk=pd.DataFrame(jj,)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [209]:
#driver = webdriver.Chrome(executable_path="./chromedriver")
driver.get('https://glints.com/api/jobs?where=%7B%22type%22:%7B%22in%22:[%22FULL_TIME%22]%7D,%22isPublic%22:true,%22isRemote%22:false,%22status%22:%22OPEN%22%7D&SearchTerm=data&include=JobSalaries,+City,+Country,+Company&order=updatedAt+DESC&limit=30&offset=30')
sleep(1)
html = driver.page_source.lower().strip()
soup = BeautifulSoup(html, 'tml')
#for x in soup.findAll('a',{'class':'turnstilelink slnounderline'}):
    #for y in x.findAll('span',{'class':'slnounderline'}):
    #print(x)
#print(soup)

FeatureNotFound: Couldn't find a tree builder with the features you requested: tml. Do you need to install a parser library?

In [218]:
import scrapy

In [217]:
Selector(text=html).xpath('')

ValueError: XPath error: Invalid expression in 

In [203]:
soup

<html xmlns="http://www.w3.org/1999/xhtml"><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">{"links":{"jobs.applications":"/jobs/{jobs.id}/applications","jobs.bookmarks":"/jobs/{jobs.id}/bookmarks","jobs.jobsalaries":"/jobs/{jobs.id}/jobsalaries","jobs.city":"/cities/{jobs.cityid","jobs.company":"/companies/{jobs.companyid","jobs.country":"/countries/{jobs.countrycode","jobs.jobcategory":"/jobcategories/{jobs.jobcategoryid","jobs.creator":"/users/{jobs.creatorid","jobs.reviewer":"/users/{jobs.reviewerid","jobs.groups":"/jobs/{jobs.id}/groups","jobs.jobtitles":"/jobs/{jobs.id}/jobtitles","jobs.skills":"/jobs/{jobs.id}/skills","jobs.tags":"/jobs/{jobs.id}/tags"},"data":[{"revieweddate":"2018-09-18t00:00:00.000z","id":"316f2d6d-f513-448c-a03d-2b0693f94614","title":"data engineer","status":"open","deadline":null,"startdate":null,"enddate":null,"bannerpic":"a115eef62eef8d27c9f84759fd4cf26b.jpg","intro":null,"location":null,"type":"full_time","durationlegacy":nul

In [205]:
import json

In [210]:
json.loads(soup)

TypeError: the JSON object must be str, bytes or bytearray, not 'BeautifulSoup'

In [194]:
for x in soup.findAll('a',{'class':'sc-bmvaic czzhoc'}):
    print(x.get('href'))
    print('-'*80)

/opportunities/jobs/data-scientist/4a5031ca-2f35-4e39-9c2b-d3f6dcc3ea1f
--------------------------------------------------------------------------------
/opportunities/jobs/data-engineer/0f586944-116a-4e71-9328-766204417c16
--------------------------------------------------------------------------------
/opportunities/jobs/data-scientist/797bbfb1-197d-4a47-9bd0-e9d2d7e2688f
--------------------------------------------------------------------------------
/opportunities/jobs/data-science-consultant/8162201f-2c64-4111-b11b-35e0a30938d9
--------------------------------------------------------------------------------
/opportunities/jobs/data-engineer/97ab33c5-7cf5-4ba4-abb1-5d8b97172c78
--------------------------------------------------------------------------------
/opportunities/jobs/senior-data-scientist/3169ce6a-25e2-4c47-8e19-0ad919118380
--------------------------------------------------------------------------------
/opportunities/jobs/manufacturing-data-specialist/2698ecb4-c018-4b6b

In [177]:
Selector(text=HTML).xpath('//div').extract()

['<div id="accessibilitybanner"><span id="accessibilitytext">skip to <!-- this is translated before reaching this state --><a id="skiptojobs" class="accessibilitymenu" href="#jobpostingsanchor">job postings</a>, <!-- this is translated before reaching this state --><a id="skiptosearch" class="accessibilitymenu" href="#what">search</a></span><a id="accessibilityclose">close</a></div>',
 '<div id="g_nav" data-tn-section="globalnav" role="navigation" class="left">\n    <table width="100%" cellpadding="0" cellspacing="0">\n        <tr>\n            <td nowrap><style type="text/css">\n#p_nav a.selected { font-weight: bold; text-decoration:none; color: #000 !important; }\n</style>\n\n<div id="p_nav"><span class="navbi"><a href="/" title="jobs" id="jobslink" class="selected">find jobs</a></span><span class="navbi"><a href="http://www.indeed.com/resumes?co=sg&amp;sp=0&amp;isid=find-resumes-sg&amp;ikw=serptop&amp;hl=en" rel="nofollow" id="rezlink">find resumes</a></span><span class="navbi"><a h

In [444]:
review

# Example

In [2]:
'''
https://www.indeed.com.sg/jobs?q=data+analytics&l=Singapore
which is equivalent to https://www.indeed.com.sg/jobs?q=data+analytics&l=Singapore&start=0
'''
url = 'https://www.indeed.com.sg/jobs?q=data+analytics&l=Singapore&start=10'

In [203]:
response = requests.get("https://www.jobstreet.com.sg/en/job-search/job-vacancy.php?key=data+science&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=2&src=16&ojs=10")
HTML_low=response.text.lower()
Selector(text=HTML_low).xpath('//html').extract()

['<html lang="en">\n<head>\n<meta charset="utf-8">\n<title>sorry, unable to access page...</title>\n<style>\nbody{font-family: arial; margin:0px; padding:0px;}\nh1{font-size: 1.7em}\n.logo{padding-top:80px;}\n</style>\n</head>\n<body>\n<center>\n<div class="logo"><img src="http://www.jobstreet.com/images/jobstreet-logo-global.png" border="0"></div>\n\n<h1>sorry, unable to access page...</h1>\nyou are entering jobstreet.com through an unauthorised source.<br>\n\nplease visit our website at <a id="jobstreet.com" href="http://www.jobstreet.com" title="jobstreet.com">www.jobstreet.com</a><span class="text-muted text-center"> </span> to apply for this job. we apologise for any inconvenience caused.<br><br>\nfor any queries, please email us at <a href="/cdn-cgi/l/email-protection#d7beb9b1b897bdb8b5a4a3a5b2b2a3f9b4b8ba"><span class="__cf_email__" data-cfemail="b0d9ded6dff0dadfd2c3c4c2d5d5c49ed3dfdd">[email\xa0protected]</span><br><br>\n\n   \n\n<div style="position: absolute; bottom: 5px; bac

In [3]:
response = requests.get(url)
HTML=response.text

In [436]:
response.status_code

200

In [10]:
"//Users/sevenlamps/Desktop/week-06/labs/python-webscraping_opentable-lab-master/chromedriver/chromedriver"

'//Users/sevenlamps/Desktop/week-06/labs/python-webscraping_opentable-lab-master/chromedriver/chromedriver'

In [438]:
'''
Use xpath to find jk ID to address different Job details
'''
response = requests.get("https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0")
HTML_low=response.text.lower()
vjk=Selector(text=HTML_low).xpath('//div/@data-jk').extract()

In [ ]:
    for x in soup.findAll('a',{'class':'turnstilelink slnounderline'}):
        for y in x.findAll('span',{'class':'slnounderline'}):
            print(y)
            print('---')

In [439]:
'''
5 dynamic vjk id, need to be figured out
'''
vjk

['b4fd692261af3cdc',
 '7d168700d0d4c814',
 'ca8ecb5cda5448b9',
 '9656c51b6a118a89',
 '973522c17a400942',
 'fe6aad76f627685e',
 'd9537561cb427fa5',
 '1d5dadcb103d3393',
 '7ef04bd41b6899ad',
 '6500eabbb9ffbcd6',
 'bf9c0d23d6d3682c',
 '987a42156bc5d69a',
 'f2023c537f67e56d',
 '2d026896c24a46b4',
 'eeb18fbc7503b624',
 '81ba4ee03fbdc88f']

In [76]:
url_chk="https://www.indeed.com.sg/jobs?q=data&l=Singapore&filter=0&start=0&vjk=7d168700d0d4c814"
xx=requests.get(url_chk)
yy=xx.text.lower()
#Selector(text=yy).xpath('//td').extract()
#zz=BeautifulSoup(yy,'html.parser')
#zz.findAll('div',{'id':'vjs-jobtitle'})

In [39]:
driver = webdriver.Chrome(executable_path="//Users/sevenlamps/Desktop/week-06/labs/python-webscraping_opentable-lab-master/chromedriver/chromedriver")
driver.get("https://www.indeed.com.sg/jobs?q=data%20%245000&l=Singapore&vjk=1c347edf2e781484")
# Wait one second.
sleep(1)
# Grab the page source.
html = driver.page_source.lower().strip()

In [6]:
zz=BeautifulSoup(html, 'lxml')

In [7]:
for x in zz.findAll('div',{'id':'vjs-jobtitle'}):
    print(x.text)

data engineer/ data analyst/ data visualization


In [323]:
for x in zz.findAll('span',{'id':'vjs-cn'}):
    print(x.text)

aryan search pte ltd


In [433]:
post_date=''
for x in zz.findAll('span',{'class':'date'}):
    post_date=x
print(post_date)

<span class="date">9 hours ago</span>


In [419]:
chk=[]

In [452]:
for i in vjk:
    scraping_finish_url = scraping_start_url + i
    driver.get(scraping_finish_url)
    sleep(1)
    html = driver.page_source.lower().strip()
    soup = BeautifulSoup(html, 'lxml')
    #div[@class='sjcl']/div[1]
    #for n in soup.findAll('div',{'class':'sjcl'}):
    for x in soup.findAll('a',{'class':'turnstilelink slnounderline'}):
        for y in x.findAll('span',{'class':'slnounderline'}):
            print(y)
            print('---')

In [109]:
df_review=pd.DataFrame(columns=['vjk_id','review_counts'])
temp_vjk_id=[]
temp_review=[]
for x in zz.findAll('a',{'class':'turnstilelink slnounderline '}):
    temp_vjk_id.append(str(x).split('fromjk=')[1].split('&')[0])
    temp_review.append(x.text.strip())
    print(str(x).split('fromjk=')[1].split('&')[0])
    print(x.text.replace('/n',''))
    print('-'*40)
df_review['vjk_id']=temp_vjk_id
df_review['review_counts']=temp_review

892ca1f00cc44f4c


12,287 reviews
----------------------------------------
b9820dc507b20c1d


12,287 reviews
----------------------------------------
67ff6cb64963b64f


12,287 reviews
----------------------------------------
f9c8b2e666226dc8


12,287 reviews
----------------------------------------
83b25e5aaa38374b


2 reviews
----------------------------------------
0c35b7894f99b9a2


13,412 reviews
----------------------------------------
7ac184977c9ee688


2 reviews
----------------------------------------
d046398e74826791


7,506 reviews
----------------------------------------
700ddb751d8e7f37


12,287 reviews
----------------------------------------
3136c8b61c9249d4


12,287 reviews
----------------------------------------


In [110]:
df_review

,vjk_id,review_counts
0,892ca1f00cc44f4c,"12,287 reviews"
1,b9820dc507b20c1d,"12,287 reviews"
2,67ff6cb64963b64f,"12,287 reviews"
3,f9c8b2e666226dc8,"12,287 reviews"
4,83b25e5aaa38374b,2 reviews
5,0c35b7894f99b9a2,"13,412 reviews"
6,7ac184977c9ee688,2 reviews
7,d046398e74826791,"7,506 reviews"
8,700ddb751d8e7f37,"12,287 reviews"
9,3136c8b61c9249d4,"12,287 reviews"


In [46]:
'''
review
'''
for x in zz.findAll('a',{'class':'turnstilelink slnounderline '}):
    for y in x.findAll('span',{'class':'slnounderline'}):
    #for y in x.findAll('span',{'class':'slnounderline'}):
    print(x)

<a class="turnstilelink slnounderline " data-tn-element="reviewstars" data-tn-variant="cmplinktst2" href="/cmp/kelly-services/reviews" onmousedown="this.href = appendparamsonce(this.href, '?campaignid=cmplinktst2&amp;from=serp&amp;jt=medical+sales+representative+%28%245000%29&amp;fromjk=892ca1f00cc44f4c&amp;jcid=88eb793d84c5db06');" rel="noopener" target="_blank" title="kelly services singapore reviews">
<span class="ratings"><span class="rating" style="width:44.4px"><!-- --></span></span>
<span class="slnounderline">12,287 reviews</span></a>
<a class="turnstilelink slnounderline " data-tn-element="reviewstars" data-tn-variant="cmplinktst2" href="/cmp/kelly-services/reviews" onmousedown="this.href = appendparamsonce(this.href, '?campaignid=cmplinktst2&amp;from=serp&amp;jt=marketing+specialist+%5c%2f+6+months+contract+%5c%2f+amk+%5c%2f+%245000&amp;fromjk=b9820dc507b20c1d&amp;jcid=88eb793d84c5db06');" rel="noopener" target="_blank" title="kelly services singapore reviews">
<span class="r

In [381]:
chk

[]

In [327]:
job_description=''
for x in zz.findAll('div',{'id':"vjs-desc"}):
    for y in x.findAll('li'):
        job_description+=y.text.strip()

In [328]:
job_description

''

In [384]:
#job_description=''
for x in zz.findAll('div',{'id':"vjs-desc"}):
    job_description=''
    for y in x.findAll('p'):
        job_description+=","
        job_description+=y.text.strip()

In [385]:
job_description

',we are hiring for one of the position of data engineering with one of the leading client multinational company into consulting, technology and operations services based out in singapore.\n\nsalary: sgd 5000 – sgd 8000,location – singapore,12 months renewable contract,job description:,data engineering:,minimum 3 years of experience in sql queries and procedures including strong knowledge of optimization/tuning,excellent python/ java/ scala programming skills to cleanse, manipulate, and present data,experience with big data technologies like hdfs, hive, sqoop, spark,experience in real time streaming/processing using kafka,familiar with shell scripting,experience with cloud, preferably aws (ec2, lambda, kinesis, s3, rds, emr etc.),minimum 2 years of experience for the design and development of etl process including fine tuning and optimization.,experience in nosql databases will be advantageous.,interested, please send over your resume to recruitment at aryansearch dot com or call at t 

In [331]:
job_description=''
for x in zz.findAll('div',{'id':"vjs-desc"}):
    job_description+=x.text.strip()

In [332]:
job_description

'we are hiring for one of the position of data engineering with one of the leading client multinational company into consulting, technology and operations services based out in singapore.\n\nsalary: sgd 5000 – sgd 8000\nlocation – singapore\n12 months renewable contract\n\njob description:\n\ndata engineering:\nminimum 3 years of experience in sql queries and procedures including strong knowledge of optimization/tuning\nexcellent python/ java/ scala programming skills to cleanse, manipulate, and present data\nexperience with big data technologies like hdfs, hive, sqoop, spark\nexperience in real time streaming/processing using kafka\nfamiliar with shell scripting\nexperience with cloud, preferably aws (ec2, lambda, kinesis, s3, rds, emr etc.)\nminimum 2 years of experience for the design and development of etl process including fine tuning and optimization.\nexperience in nosql databases will be advantageous.\n\ninterested, please send over your resume to recruitment at aryansearch dot

In [13]:
driver = webdriver.Chrome(executable_path="//Users/sevenlamps/Desktop/week-06/labs/python-webscraping_opentable-lab-master/chromedriver/chromedriver")
driver.get("https://www.indeed.com.sg/jobs?q=data%20%244000&l=Singapore&start=20")
# Wait one second.
sleep(1)
# Grab the page source.
html = driver.page_source

In [14]:
html_lower=html.lower().strip()

In [15]:
vjk_list=Selector(text=html_lower).xpath('//div/@data-jk').extract()

In [16]:
'''
Use xpath to find jk ID to address different Job details
'''
vjk_list=Selector(text=html_lower).xpath('//div/@data-jk').extract()
vjk_list
driver.get("https://www.indeed.com.sg/jobs?q=data%20%244000&l=Singapore&start=20&vjk=c6592d4325692dc0")
sleep(1)
html = driver.page_source
html_soup = BeautifulSoup(html, 'lxml')


In [205]:
driver.get('https://www.jobstreet.com.sg/en/job-search/job-vacancy.php?key=data+science&area=1&option=1&job-source=1%2C64&classified=1&job-posted=0&sort=2&order=0&pg=2&src=16&ojs=10')
sleep(1)
html = driver.page_source.lower().strip()
soup = BeautifulSoup(html, 'lxml')

In [206]:
soup.findAll('div')

[<div class="wrap" id="main">
 <div class="mobile-navbar-fixed-top ">
 <div>
 <div class="navbar-default navbar-static">
 <div class="container full-width">
 <div class="navbar-header" id="mobile_search_bar">
 <button class="navbar-toggle" data-canvas="body" data-target=".navbar-collapse" data-toggle="collapse" id="header-responsive-button" onclick="$('#mobile_search_bar').removeclass('active');" type="button">
 <span class="icon-bar"></span>
 <span class="icon-bar"></span>
 <span class="icon-bar"></span>
 </button>
 <span class="jslogo jslogo-mobile visible-xs"></span>
 <!--mobile view search bar toggle button-->
 <div class="search-toggle">
 <a href="#"><i class="icon-search navbar-brand visible-xs" id="mobile_search" onclick="$('#mobile_search_bar_menu').removeclass('in');"></i></a>
 </div>
 <div class="search-bar toggle">
 <div class="container visible-xs">
 <div class="col-xs-12">
 <form action="https://www.jobstreet.com.sg/en/job-search/job-vacancy.php" id="frmsearch_mobile" meth

In [207]:
driver.get('https://www.jobstreet.com.sg/en/job/senior-analyst-office-for-insights-analytics-6806009?fr=21&src=16&searchRequestToken=30a752c6-8db7-4713-ec88-f84d26046f6f&sectionRank=21')
sleep(1)
html = driver.page_source.lower().strip()
soup = BeautifulSoup(html, 'lxml')

In [211]:
'''

'''
soup.findAll('span')

[<span class="icon-bar"></span>,
 <span class="icon-bar"></span>,
 <span class="icon-bar"></span>,
 <span class="jslogo jslogo-mobile visible-xs"></span>,
 <span class="icon-search visible-xs" id="mobile_search" onclick="$('#mobile_search_bar').toggleclass('active');"></span>,
 <span class="input-group-btn">
 <button class="btn btn-default" type="submit">search</button>
 </span>,
 <span>|</span>,
 <span class="input-group-btn"><button class="btn btn-default" type="submit"><span class="icon-search"></span></button></span>,
 <span class="icon-search"></span>,
 <span class="main_desc_icon">
 <span class="icon-dollar-sign icon_left salary"></span>
 </span>,
 <span class="icon-dollar-sign icon_left salary"></span>,
 <span class="main_desc_icon">
 <span class="icon-briefcase icon_left experience"></span>
 </span>,
 <span class="icon-briefcase icon_left experience"></span>,
 <span id="years_of_experience">
 			min 4 years (junior executive) 
 		</span>,
 <span class="main_desc_icon" id="work_

In [434]:
driver.get('https://www.mycareersfuture.sg/search?search=Data%20Analytics&sortBy=new_posting_date&page=1')
sleep(1)
html2 = driver.page_source.lower().strip()
soup2 = BeautifulSoup(html2, 'lxml')

In [378]:
soup2.findAll('a',)

[<a class="no-underline pointer" href="/" id="logo"><i class="ml2 mainnavigation__cclogo___12jav"></i></a>,
 <a href="http://www.wsg.gov.sg/adapt-and-grow.html" target="_blank"><i class="ml2 mainnavigation__wsglogo___3wdo0"></i></a>,
 <a aria-current="true" class="f5 fw6 black-60 no-underline pv3 ph2 ph3-ns dib relative browsermenu__browser-menu-item___3mdpt browsermenu__active___xl1dy" href="/" id="navbar-search-jobs">search jobs</a>,
 <a class="f5 fw6 black-60 no-underline pv3 ph2 ph3-ns dib relative browsermenu__browser-menu-item___3mdpt" href="https://content.mycareersfuture.sg" id="navbar-careers-toolkit" rel="noopener" target="_blank">careers toolkit</a>,
 <a class="f5 fw6 black-60 no-underline pv3 ph2 ph3-ns dib relative browsermenu__browser-menu-item___3mdpt browsermenu__icon-login___1jffr" href="#" id="navbar-login">log in</a>,
 <a class="f5 underline pointer"><span class="brand fw5">show 162 jobs with government support</span></a>,
 <a class="bg-white mb3 w-100 dib v-top pa3 

# SG_job_bank <=Gov web

In [569]:
col_all

['title',
 'intro',
 'type',
 'min_year_exp',
 'max_year_exp',
 'is_public',
 'num_of_hires',
 'is_remote',
 'job_description',
 'country_code',
 'sal_mode',
 'sal_min',
 'sal_max',
 'sal_currency_code',
 'comp_name',
 'comp_tagline',
 'comp_verification',
 'comp_is_hiring',
 'comp_local_office',
 'comp_description',
 'country_name',
 'country_demonym',
 'country_currency',
 'country_language']

In [571]:
sg_col=['skills','company_name','job_title','job_type','job_seniority','job_category',
       'job_min_salary','job_max_salary','job_sal_type','job_description_p','job_description_li',
       'job_requirement_p','job_requirement_li']

In [573]:
df_sg_job_bank=pd.DataFrame(columns=sg_col)

In [574]:
df_sg_job_bank.skills=jb_skills

In [575]:
df_sg_job_bank.company_name=jb_comp_name

In [576]:
df_sg_job_bank.job_title=jb_job_title

In [577]:
df_sg_job_bank.job_type=jb_job_type

In [578]:
df_sg_job_bank.job_seniority=jb_seniority

In [579]:
df_sg_job_bank.job_category=jb_job_cat

In [581]:
df_sg_job_bank.job_min_salary=jb_min_sal

In [580]:
df_sg_job_bank.job_max_salary=jb_max_sal

In [582]:
df_sg_job_bank.job_sal_type=jb_sal_type

In [583]:
df_sg_job_bank.job_description_p=jb_descript_p

In [584]:
df_sg_job_bank.job_description_li=jb_descript_li

In [585]:
df_sg_job_bank.job_requirement_p=jb_require_p

In [586]:
df_sg_job_bank.job_requirement_li=jb_require_li

In [588]:
df_sg_job_bank.to_csv('data_myfulturcareer.csv')

In [568]:
import random
pg_list=range(0,19)
#pg_list=[2]
url_start='https://www.mycareersfuture.sg/search?search=Data%20Analytics&sortBy=new_posting_date&page='
jb_skills=[]
jb_comp_name=[]
jb_job_title=[]
jb_job_type=[]
jb_seniority=[]
jb_job_cat=[]
jb_min_sal=[]
jb_max_sal=[]
jb_sal_type=[]
jb_descript_p=[]
jb_descript_li=[]
jb_require_p=[]
jb_require_li=[]
for page in pg_list:
    print(page)
    job_id_list=[]
    sl=random.randint(6,12)
    url_finish=url_start+str(page)
    #print(url_finish)
    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(url_finish)
    sleep(sl)
    HTML = driver.page_source.lower().strip().replace('<br />','\n')
    driver.close()

    job_id_list=[x for x in Selector(text=HTML).xpath('//a/@href').extract() if '/job/' in x]
    #print(job_id_list)
    for job in job_id_list:
        job_url_start='https://www.mycareersfuture.sg'
        job_url_final=job_url_start+job
        print(job_url_final)
        driver = webdriver.Chrome(executable_path="./chromedriver")
        driver.get(job_url_final)
        wait=random.randint(15,25)
        sleep(wait)
        html3 = driver.page_source.lower().strip()
        html3=html3.replace('<br />','\n')
        soup3 = BeautifulSoup(html3, 'lxml')
        driver.close()
        
        temp=[]
        if len(soup3.findAll('div',{'class':'pill-button dib ma1'}))==0:
            temp.append(None)
        else:
            for x in soup3.findAll('div',{'class':'pill-button dib ma1'}):
                temp.append(x.text)
        jb_skills.append(temp)
        
        if len(soup3.findAll('p',{'class':'f6 fw6 mv0 black-80 mr2 di ttu'}))==0:
            jb_comp_name.append(None)
        else:
            for x in soup3.findAll('p',{'class':'f6 fw6 mv0 black-80 mr2 di ttu'}):
                jb_comp_name.append(x.text)
                
        if len(soup3.findAll('h1',{'id':'job_title'}))==0:
            jb_job_title.append(None)
        else:
            for x in soup3.findAll('h1',{'id':'job_title'}):
                jb_job_title.append(x.text)
        
        if len(soup3.findAll('p',{'id':'employment_type'}))==0:
            jb_job_type.append(None)
        else:
            for x in soup3.findAll('p',{'id':'employment_type'}):
                jb_job_type.append(x.text)
                
        if len(soup3.findAll('p',{'id':'seniority'}))==0:
            jb_seniority.append(None)
        else:
            for x in soup3.findAll('p',{'id':'seniority'}):
                jb_seniority.append(x.text)
        
        if len(soup3.findAll('p',{'id':'job-categories'}))==0:
            jb_job_cat.append(None)
        else:
            for x in soup3.findAll('p',{'id':'job-categories'}):
                jb_job_cat.append(x.text)
                
        if len(soup3.findAll('span',{'class':'f5-5 i fw4 gray'}))!=0:
            jb_min_sal.append(None)
        else:
            jb_min_sal.append(soup3.findAll('span',{'class':'dib'})[2].text)
            
        if len(soup3.findAll('span',{'class':'f5-5 i fw4 gray'}))!=0:
            jb_max_sal.append(None)
        else:
            jb_max_sal.append(soup3.findAll('span',{'class':'dib'})[3].text.replace('to',''))    
            
        if len(soup3.findAll('span',{'class':'f5-5 i fw4 gray'}))!=0:
            jb_sal_type.append(None)
        else:
            for x in soup3.findAll('span',{'class':'salary_type dib f5 fw4 black-60 pr1 i pb'}):
                jb_sal_type.append(x.text)
        
        temp=[]
        for x in soup3.findAll('div',{'id':'job_description'}):
            for y in x.findAll('div',{'id':'content'}):
                if len(y.findAll('p'))==0:
                    temp.append(None)
                else:           
                    for z in y.findAll('p'):
                        temp.append(z.text.replace('   ',""))
        jb_descript_p.append(temp)

        temp=[]
        for x in soup3.findAll('div',{'id':'job_description'}):
            for y in x.findAll('div',{'id':'content'}):
                if len(y.findAll('li'))==0:
                    temp.append(None)
                else:           
                    for z in y.findAll('li'):
                        temp.append(z.text.replace('   ',""))
        jb_descript_li.append(temp)
        
        temp=[]
        for x in soup3.findAll('div',{'id':'requirements'}):
            for y in x.findAll('div',{'id':'content'}):
                if len(y.findAll('p'))==0:
                    temp.append(None)
                else:           
                    for z in y.findAll('p'):
                        temp.append(z.text.replace('   ',""))
        jb_require_p.append(temp)

        temp=[]
        for x in soup3.findAll('div',{'id':'requirements'}):
            for y in x.findAll('div',{'id':'content'}):
                if len(y.findAll('li'))==0:
                    temp.append(None)
                else:           
                    for z in y.findAll('li'):
                        temp.append(z.text.replace('   ',""))
        jb_require_li.append(temp)
        
       

0
https://www.mycareersfuture.sg/job/data-analytics-lead-traveloka-services-aa9bd39f7c57d70ffa1e4e44799bd305
https://www.mycareersfuture.sg/job/svp-infrastructure-lead-group-consumer-banking-big-data-analytics-technology-to-dbs-bank-7064a72ee22a43eb2be59f72b37df48a
https://www.mycareersfuture.sg/job/head-accountant-generals-department-cb19026c04d291f44112432e6550e2e9
https://www.mycareersfuture.sg/job/data-analyst-data-transformation-imo-robert-walters-bc6d630c450ff0f492bae277bfdfc3be
https://www.mycareersfuture.sg/job/data-manager-data-management-aia-singapore-d58b097c6d6b93c7be3d761acc1ba25d
https://www.mycareersfuture.sg/job/senior-assistant-director-survey-design-analytics-contract-ministry-manpower-3fcd07c240b0bafa917fc5ee3263f240
https://www.mycareersfuture.sg/job/research-fellow-national-university-singapore-fdfb32e7c29c1da01af0bdedaa774c2d
https://www.mycareersfuture.sg/job/mplatform-head-biddable-search-apac-groupm-asia-pacific-holdings-6d7a7f93d1a4122cb1052d3f396488e7
https:/

https://www.mycareersfuture.sg/job/senior-recruitment-consultant-hudson-global-resources-ad18bc7a764bc31a123f74a18318a590
https://www.mycareersfuture.sg/job/data-architect-enterprise-architecture-c698e019ddf43fedec76b8ffcc8c4ad6
https://www.mycareersfuture.sg/job/data-analytics-associate-optimum-solutions-d7f5a42913fed23b9d7f0b7dd7d11f0b
https://www.mycareersfuture.sg/job/data-analytics-excel-macro-tableau-sql-optimum-solutions-4ba45b93407e9331a7794d6a55e9c039
https://www.mycareersfuture.sg/job/ts-manager-data-analytics-aspac-johnson-johnson-dfdfd9c26c505d5d2cea266bb6b9ecca
https://www.mycareersfuture.sg/job/manager-logistics-transportation-data-analytics-services-product-line-owner-aspac-johnson-johnson-3856a37b1e400ff8b7e35460ed09591d
https://www.mycareersfuture.sg/job/data-scientist-cloudstream-asia-pacific-e51a03f0ad45d72d8982553ecb6db959
https://www.mycareersfuture.sg/job/restaurant-manager-social-room-concepts-24335fac05f3c07451b287a985fb5cdd
https://www.mycareersfuture.sg/job/cl

https://www.mycareersfuture.sg/job/regional-data-analyst-grabtaxi-holdings-e5cb88da60f41c0088a75c217e098045
https://www.mycareersfuture.sg/job/senior-lead-specialist-integrated-health-information-systems-7f603b1a66db939cd4b6f346e97fae10
https://www.mycareersfuture.sg/job/lead-specialist-integrated-health-information-systems-b99a347492152bb6f42a4b1c66a9ace3
https://www.mycareersfuture.sg/job/senior-specialist-integrated-health-information-systems-e4b92cc57e7b4e21fec72e7df3c13f4a
https://www.mycareersfuture.sg/job/senior-specialist-integrated-health-information-systems-c5a74b1c5209f03d096775ae53fb5504
https://www.mycareersfuture.sg/job/specialist-integrated-health-information-systems-17be05ac0710ad5f2a11e894335da90a
https://www.mycareersfuture.sg/job/research-associate-singapore-university-technology-design-66966ce176908e3c7e23e3d0b23982e3
https://www.mycareersfuture.sg/job/scientist-i2r-astar-research-entities-da9621e8fa4c24260c7aa447b8b0d216
https://www.mycareersfuture.sg/job/scientist

https://www.mycareersfuture.sg/job/executive-senior-executive-ministry-social-family-development-8af3c2396b21deea37c721de9e77fb08
https://www.mycareersfuture.sg/job/manager-citygrass-bca0a3f22d98acafda1779c2346f018b
https://www.mycareersfuture.sg/job/senior-financial-analyst-allegis-group-singapore-5d761e4470b5139b0e475cb030aac4df
https://www.mycareersfuture.sg/job/data-engineer-people-profilers-c751566bb80319a39400f1f21ffd396b
https://www.mycareersfuture.sg/job/research-engineer-i%c2%b2r-astar-research-entities-cbbcac4924c4fee3837e8e5731d33d01
https://www.mycareersfuture.sg/job/product-analyst-transferwise-singapore-a0d1b6d352a1051c4da9004cbe30439f
https://www.mycareersfuture.sg/job/assistant-director-business-services-integrated-business-services-division-skillsfuture-singapore-agency-3322b462d0257398dfd34472a2b1e97a
https://www.mycareersfuture.sg/job/risk-assurance-senior-manager-pricewaterhousecoopers-risk-services-5d11edc64d14dc8646792f8345fcfac4
11
https://www.mycareersfuture.sg/

https://www.mycareersfuture.sg/job/professional-development-manager-international-baccalaureate-organization-8db7e72add9791424184ce50a33550df
https://www.mycareersfuture.sg/job/assistant-vice-president-data-scientist-76987ba43dbc673083f14e54c4022f00
https://www.mycareersfuture.sg/job/ot-crs-apac-real-estate-strategy-head-citibank-na-9a4f5322e3e01ccb866361459d69f49c
https://www.mycareersfuture.sg/job/full-stack-developer-mesh-bio-aef6b2bf66fa3997d8d7464ff06c7187
https://www.mycareersfuture.sg/job/engineering-manager-transport-redmart-96d99dff99a97f6860afd64297b4222f
https://www.mycareersfuture.sg/job/software-engineer-singapore-google-asia-pacific-adcf9ae11203006fa3aafcda29ba0f50
https://www.mycareersfuture.sg/job/digital-marketing-executives-opus-kinetic-af91eea694f1e4f87cfcf160eb35f8c2
https://www.mycareersfuture.sg/job/junior-finance-analyst-mullenlowe-singapore-cfecc3370c675383bb08fd909d2660b1
https://www.mycareersfuture.sg/job/senior-insights-analyst-aimia-proprietary-loyalty-singa

https://www.mycareersfuture.sg/job/manager-senior-manager-maritime-port-authority-singapore-cd6aff623d86b0588668343c21856949
https://www.mycareersfuture.sg/job/manager-intelligence-analytics-private-education-services-division-skillsfuture-singapore-agency-fe4eb342bf492f020a54d6b3a436ad12
https://www.mycareersfuture.sg/job/senior-executive-data-management-singapore-land-authority-be51f608afd50e365ac2c824d2e34762
https://www.mycareersfuture.sg/job/office-human-resources-manager-assistant-manager-republic-polytechnic-5d87e6f438d973d99372e3fa86693f0c
https://www.mycareersfuture.sg/job/principal-manager-digital-product-project-manager-partners-operations-division-workforce-singapore-agency-5dc69a1d3f6e8e1a9ea1188a487c96f5
https://www.mycareersfuture.sg/job/school-sports-health-leisure-assistant-manager-deputy-manager-manager-republic-polytechnic-a7c1ec9195cfb1e23808a0ab8d1198d0
https://www.mycareersfuture.sg/job/manager-event-operations-partners-operations-division-workforce-singapore-agen

In [567]:
import random
random.randint(15,25)

16

In [518]:
Selector(text=HTML).xpath('//a/@href').extract()

['/',
 'http://www.wsg.gov.sg/adapt-and-grow.html',
 '/',
 'https://content.mycareersfuture.sg',
 '/docs/mycareersfuture_sg_user_faqs.pdf',
 'terms-of-use',
 'privacy-policy',
 'survey',
 'https://portal.ssg-wsg.gov.sg',
 'https://www.myskillsfuture.sg/content/portal/en/index.html?spcplogin=true',
 '']

In [ ]:
col_job=['title','intro','type','min_year_exp','max_year_exp','is_public','num_of_hires','is_remote','job_description','country_code','sal_mode','sal_min','sal_max','sal_currency_code']
col_comp=['comp_name','comp_tagline','comp_verification','comp_is_hiring','comp_local_office','comp_description']
col_country=['country_name','country_demonym','country_currency','country_language']
col_all=col_job+col_comp+col_country
df_glints_data=pd.DataFrame(columns=col_all)

In [435]:
temp=Selector(text=html2).xpath('//a/@href').extract()

In [436]:
[x for x in temp if 'job' in x]

['https://content.mycareersfuture.sg/how-to-land-a-job-in-healthcare-3-tips-from-an-industry-veteran/']

In [535]:
#driver.close()
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.get('https://www.mycareersfuture.sg/job/school-infocomm-research-staff-republic-polytechnic-bdb3a8b22365e72310e2edc0fbfc737a')
sleep(1)
html3 = driver.page_source.lower().strip()
html3=html3.replace('<br />','\n')
sleep(1)
driver.close()
soup3 = BeautifulSoup(html3, 'lxml')

In [536]:
'''
Required skills
'''
temp=[]
if len(soup3.findAll('div',{'class':'pill-button dib ma1'}))==0:
    print(None)
else:
    for x in soup3.findAll('div',{'class':'pill-button dib ma1'}):
        temp.append(x.text)

In [538]:
'''
company
'''
if len(soup3.findAll('p',{'class':'f6 fw6 mv0 black-80 mr2 di ttu'}))==0:
    print(None)
else:
    for x in soup3.findAll('p',{'class':'f6 fw6 mv0 black-80 mr2 di ttu'}):
        print(x.text)

republic polytechnic


In [539]:
'''
job title
'''
if len(soup3.findAll('h1',{'id':'job_title'}))==0:
    print(None)
else:
    for x in soup3.findAll('h1',{'id':'job_title'}):
        print(x.text)

school of infocomm - research staff (text analytics)


In [540]:
'''
job type
'''
if len(soup3.findAll('p',{'id':'employment_type'}))==0:
    print(None)
else:
    for x in soup3.findAll('p',{'id':'employment_type'}):
        print(x.text)

contract


In [541]:
'''
seniority
'''
if len(soup3.findAll('p',{'id':'seniority'}))==0:
    print(None)
else:
    for x in soup3.findAll('p',{'id':'seniority'}):
        print(x.text)

None


In [542]:
'''
job-categories
'''
if len(soup3.findAll('p',{'id':'job-categories'}))==0:
    print(None)
else:
    for x in soup3.findAll('p',{'id':'job-categories'}):
        print(x.text)

education and training, public / civil service


In [543]:
'''
min sal
'''
if len(soup3.findAll('span',{'class':'dib'})[2].text)==0:
    print(None)
else:
    print(soup3.findAll('span',{'class':'dib'})[2].text)

None


In [546]:
if len(soup3.findAll('span',{'class':'f5-5 i fw4 gray'}))!=0:
    print(None)
else:
    print(soup3.findAll('span',{'class':'dib'})[2].text)

None


In [549]:
len(soup3.findAll('span',{'class':'f5-5 i fw4 gray'}))

1

In [544]:
'''
max sal
'''
if len(soup3.findAll('span',{'class':'dib'})[3].text.replace('to',''))==0:
    print(None)
else:
    print(soup3.findAll('span',{'class':'dib'})[3].text.replace('to',''))

closing on 31 dec 2018


In [534]:
if len(soup3.findAll('span',{'class':'f5-5 i fw4 gray'}))!=0:
    print(None)
else:
    for x in soup3.findAll('span',{'class':'salary_type dib f5 fw4 black-60 pr1 i pb'}):
        print(x.text)

monthly


In [ ]:
'''
job_description_li
'''
for x in soup3.findAll('div',{'id':'job_description'}):
    for y in x.findAll('div',{'id':'content'}):
        if len(y.findAll('li'))==0:
            print(None)
        else:           
            for z in y.findAll('li'):
                print(z.text.replace('   ',""))
                print('-'*40)

In [511]:
'''
job_description_p
'''
for x in soup3.findAll('div',{'id':'job_description'}):
    for y in x.findAll('div',{'id':'content'}):
        if len(y.findAll('p'))==0:
            print(None)
        else:           
            for z in y.findAll('p'):
                print(z.text.replace('   ',""))
                print('-'*40)

business function
 
 group technology and operations (t&o) enables and empowers the bank with an efficient, nimble and resilient infrastructure through a strategic focus on productivity, quality & control, technology, people capability and innovation. in group t&o, we manage the majority of the bank's operational processes and inspire to delight our business partners through our multiple banking delivery channels.
 
 
 key accountabilities
----------------------------------------
responsibilities 
----------------------------------------


In [502]:
'''
job_description_li
'''
for x in soup3.findAll('div',{'id':'job_description'}):
    for y in x.findAll('div',{'id':'content'}):
        if len(y.findAll('p'))==0:
            print(None)
        else:
            for z in y.findAll('li'):
                print(z.text.replace('  ',""))
                print('-'*40)

manage the development of the internal engineering productivity tools and environments.
----------------------------------------
providing devops architecture implementation and operational support
----------------------------------------
architecture and planning for cloud deployments (private and public cloud); 
----------------------------------------
be an innovative and hands-on devops engineer capable of looking at both the technology and strategy around the platform.
----------------------------------------
future-proofing the technical environments and ensuring extreme high levels of automation, availability, scalability and resilience.
----------------------------------------
manage the development of the internal engineering productivity tools and environments.
----------------------------------------
manage processes, automation, best practices, documentation.
----------------------------------------
development and operation of continuous integration and deployment pipeline

In [359]:
'''
job_requirement_p
'''
test=[]
for x in soup3.findAll('div',{'id':'requirements'}):
    for y in x.findAll('div',{'id':'content'}):
        print(len(y.findAll('p')))
        if len(y.findAll('p'))==0:
            test.append(None)
        else:
            for z in y.findAll('p'):
                test.append(z.text.replace('  ',""))
                print(test)

0


In [361]:
'''
job_requirement_li
'''
test=[]
for x in soup3.findAll('div',{'id':'requirements'}):
    for y in x.findAll('div',{'id':'content'}):
        print(len(y.findAll('li')))
        if len(y.findAll('li'))==0:
            test.append(None)
        else:
            for z in y.findAll('li'):
                test.append(z.text.replace('  ',""))

16


In [362]:
test

['phd/masters/bachelors in computer science, computer engineering, statistics, applied mathematics, or related disciplines.\xa0',
 'excellent understanding of software engineering principles and design patterns.',
 'excellent programming skills in either python, scala, or java.',
 'in-depth understanding of data science and machine learning technologies and methodologies.',
 'good working knowledge of high performance computing, parallel data processing, and big data stack, e.g. spark and hadoop/yarn.',
 'experience to one or more commercial / open source data warehouses or data analytics systems, e.g. teradata, is a big plus.',
 'experience to one or more nosql databases is a big plus.',
 'hands-on experience in cloud platforms, e.g. aws, or containerization/ virtualization platforms, e.g. docker/kubernetes, is a big plus.',
 'experience to any data science or machine learning platform, e.g. ibm data science experience or cloudera data science workbench, is a big plus.',
 'exposure to

In [363]:
df_indeed=pd.read_csv('./indeed_job_no_salary.csv')

In [366]:
df_indeed.drop('Unnamed: 0',axis=1,inplace=True)